# Exercícios de Árvores de Decisão
## Jhonatan Guilherme O. Pereira

### 1. Dada a tabela abaixo com os Ganhos de Informação por nível, construa a árvore de decisão.

| | X1 | X2 | X3 | X4 | X5 |
| --- | --- | --- | --- | --- | --- |
| **Primeiro Nível**  | 0.030 | 0.022 | 0.031 | 0.040 | 0.015 |
| **Segundo Nível E** | 0.020 | 0.035 | 0.012 | 0.001 | 0.030 |
| **Segundo Nível D** | 0.043 | 0.032 | 0.050 | 0.032 | 0.045 |

*Valores*:

- X1 = {M (D), F (D), I (E)}
- X2 = {Alto (E), Baixo (D)}
- X3 = {Sim (D), Não (E)}
- X4 = 80..250 (Escolhido 135.2 como ponto de corte. D (>=), E (<))
- X5 = 1..100 (Escolhido 44.5 como ponto de corte. D (<), E (>=))

*Classes*:
- D = Direita
- E = Esquerda

**Quando a árvore tiver montada, complete aleatoriamente as folhas com os valores y $\epsilon$ {0, 1}.**

![](assets/decisions_tree.jpeg)

### 2 - Construa as Regras de Decisão da questão anterior.

- if *X4 >= 135.2* and *X3 == 'Sim'* então **classe = 0**;
- if *X4 >= 135.2* and *X3 == 'Não'* então **classe = 1**;
- if *X4 < 135.2* and *X2 == 'Baixo'* então **classe = 1**;
- if *X4 < 135.2* and *X2 == 'Alto'* então **classe = 0**;

### 3 - Dado o conjunto de treinamento abaixo, responda as perguntas:
| a1 | a2 | classe |
| --- | --- | --- |
| T | T | + |
| T | T | + |
| T | F | - |
| F | F | + |
| F | T | - |
| F | T | - |

*a)* **Qual é a entropia do conjunto de treinamento?**

$ent = -(p+) log_2 (p+) - (p-) log_2(p-)$\
$ent = -(3/6) log_2(3/6) - (3/6) log_2(3/6)$\
$ent = 1$

*b)* **Qual é o ganho de informação de $a_2$?**

$ganho(Exs, Atri) = entropia(Exs) - \sum_{v}\dfrac{Exs_v}{Exs}entropia(Exs_v)$\
$ent(a2, T) = -(\frac{2}{4}) log_2(\frac{2}{4}) - (\frac{2}{4}) log_2(\frac{2}{4})$\
$ent(a2, F) = -(\frac{1}{2}) log_2(\frac{1}{2}) - (\frac{1}{2}) log_2(\frac{1}{2})$\
$info(a2) = \frac{4}{6} * ent(a2, T) + \frac{2}{6} * ent(a2, F)$

### 4 - Acessar o UCI Repository e baixar duas bases de classificação. Treinar uma árvore de decisão para cada base com 80% dos dados. Os outros 20% usar para testes. Calcular a taxa de acerto do conjunto de testes de cada base de testes. Usar como modelo o notebook: 2_Árvores_de_Decisão.ipynb

**Link do UCI Repository: *https://archive.ics.uci.edu/***

In [81]:
import io, requests, zipfile # decompress the data
import glob # file manipulation
import matplotlib.pyplot as plt # data visualization
import numpy as np # data manipulation
import pandas as pd # data manipulation
from sklearn import metrics # model evaluation
from sklearn.feature_extraction.text import CountVectorizer # bag of words
from sklearn.model_selection import train_test_split # split data
from sklearn.tree import DecisionTreeClassifier # decision tree algorithm

#### a) Dataset 1: *https://archive.ics.uci.edu/dataset/780/har70*

**Descrição:**
Este Dataset contém dados de 70 pessoas que realizaram 6 atividades diferentes (andar, subir escadas, descer escadas, sentar, ficar em pé e deitar). Os dados foram coletados através de um acelerômetro e um giroscópio do celular. O objetivo é classificar as atividades de acordo com os dados coletados.

In [5]:
response = requests.get("https://archive.ics.uci.edu/static/public/780/har70.zip", stream=True)
zipfile = zipfile.ZipFile(io.BytesIO(response.content))
zipfile.extractall("./datasets")

In [34]:
glob.glob("./har70plus/*")

['./har70plus/515.csv',
 './har70plus/516.csv',
 './har70plus/512.csv',
 './har70plus/502.csv',
 './har70plus/518.csv',
 './har70plus/514.csv',
 './har70plus/511.csv',
 './har70plus/504.csv',
 './har70plus/517.csv',
 './har70plus/501.csv',
 './har70plus/509.csv',
 './har70plus/506.csv',
 './har70plus/503.csv',
 './har70plus/505.csv',
 './har70plus/513.csv',
 './har70plus/508.csv',
 './har70plus/510.csv',
 './har70plus/507.csv']

In [11]:
df = pd.concat(map(pd.read_csv, glob.glob("./har70plus/*.csv")))
df

,timestamp,back_x,back_y,back_z,thigh_x,thigh_y,thigh_z,label
0,2021-05-27 09:28:14.420,-0.950439,0.075195,-0.291260,-0.993408,0.031006,0.073975,6
1,2021-05-27 09:28:14.440,-0.967285,0.055908,-0.270996,-1.042725,0.077393,0.069580,6
2,2021-05-27 09:28:14.460,-0.997070,0.023682,-0.241455,-1.041260,0.087891,0.042725,6
3,2021-05-27 09:28:14.479,-1.009521,-0.005127,-0.227783,-0.975586,0.079834,0.058838,6
4,2021-05-27 09:28:14.500,-0.994141,-0.004150,-0.213379,-0.977051,0.052734,0.039307,6
...,...,...,...,...,...,...,...,...
120120,2021-04-15 15:10:05.059,-0.846436,-0.114746,-0.501221,-0.897949,0.110840,0.039062,1
120121,2021-04-15 15:10:05.079,-0.808594,-0.112305,-0.494141,-0.887207,0.070068,0.050049,1
120122,2021-04-15 15:10:05.099,-0.797607,-0.113770,-0.517822,-0.918213,0.034424,0.018066,1
120123,2021-04-15 15:10:05.119,-0.807861,-0.105957,-0.521240,-0.937256,0.016357,0.015381,1


The dataset contains the following annotated activities with the corresponding coding scheme:
- 1: walking
- 3: shuffling
- 4: stairs (ascending)	
- 5: stairs (descending)	
- 6: standing	
- 7: sitting	
- 8: lying

In [18]:
# the coding 2 is not valid
df[df['label'] == 2]

,timestamp,back_x,back_y,back_z,thigh_x,thigh_y,thigh_z,label


In [21]:
X = df.drop(['timestamp', 'label'], axis=1)
y = df['label']

In [22]:
X

,back_x,back_y,back_z,thigh_x,thigh_y,thigh_z
0,-0.950439,0.075195,-0.291260,-0.993408,0.031006,0.073975
1,-0.967285,0.055908,-0.270996,-1.042725,0.077393,0.069580
2,-0.997070,0.023682,-0.241455,-1.041260,0.087891,0.042725
3,-1.009521,-0.005127,-0.227783,-0.975586,0.079834,0.058838
4,-0.994141,-0.004150,-0.213379,-0.977051,0.052734,0.039307
...,...,...,...,...,...,...
120120,-0.846436,-0.114746,-0.501221,-0.897949,0.110840,0.039062
120121,-0.808594,-0.112305,-0.494141,-0.887207,0.070068,0.050049
120122,-0.797607,-0.113770,-0.517822,-0.918213,0.034424,0.018066
120123,-0.807861,-0.105957,-0.521240,-0.937256,0.016357,0.015381


In [23]:
y

0         6
1         6
2         6
3         6
4         6
         ..
120120    1
120121    1
120122    1
120123    1
120124    1
Name: label, Length: 2259597, dtype: int64

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=None)

In [27]:
model = DecisionTreeClassifier(criterion='entropy')
model.fit(X_train, y_train)

DecisionTreeClassifier(criterion='entropy')

In [32]:
result = model.predict(X_test)
acc = metrics.accuracy_score(result, y_test)
show = round(acc * 100, 2)
"Accuracy: {}%".format(show)

'Accuracy: 90.31%'

#### b) Base 2: *https://archive.ics.uci.edu/dataset/695/sundanese+twitter+dataset*

**Descrição:** Este Dataset contém dados de 2510 tweets em Sundanese (língua falada na Indonésia) classificados em 4 categorias de humor: *raiva*, *alegria*, *medo* e *tristeza*. O objetivo é classificar os tweets de acordo com a sua categoria.

In [57]:
response = requests.get("https://archive.ics.uci.edu/static/public/695/sundanese+twitter+dataset.zip", stream=True)
zip = zipfile.ZipFile(io.BytesIO(response.content))
zip.extractall("./datasets/sundanece_twitter/")

In [58]:
glob.glob("./datasets/sundanece_twitter/*")

['./datasets/sundanece_twitter/data.csv',
 './datasets/sundanece_twitter/stopwordv1.txt',
 './datasets/sundanece_twitter/test.csv']

In [64]:
train_df = pd.read_csv("./datasets/sundanece_twitter/data.csv")
train_df['label'].unique()

array(['anger', 'joy', 'fear', 'sadness'], dtype=object)

In [66]:
train_df

,label,data
0,anger,"sok geura leungit atuh sia teh corona, matak g..."
1,anger,"Nu katoel katuhu nu nyerina kenca, goblog wasitna"
2,joy,Bingah pisan patepang sareng pangerasa. Sing k...
3,fear,asa hariwang kieu.. lalakon hirup teh asa nyor...
4,anger,"Orang mana sih anying, sampis pisan. Bunuh ae ..."
...,...,...
2093,joy,Meuni ngeunah ngabarakatak eeeuy uiiiiing....!
2094,joy,"Meuni ku endah, tebeda nami na gusti nu agung,..."
2095,joy,"meuni bungah kieu euy meunang di ngalam, nuhun..."
2096,joy,"Meuni bingah ningalna ya alloh mudah"" PaS 02 k..."


In [65]:
test_df = pd.read_csv("./datasets/sundanece_twitter/test.csv")
test_df

,label,data
0,fear,meuni asa rame nya crime dimamana... kemarin j...
1,joy,Merinding ningali mic check rrq. Pernah ngasaa...
2,sadness,"mentang2 mamah ges dulur sabelah, meuni te dia..."
3,fear,meni serem atuh pas buka grup keluarga isinya ...
4,sadness,"Meni sedih udah mah ga boleh keluar, di dalem ..."
...,...,...
407,fear,Abi gawe di rs boromeus sararoak kieu:(
408,joy,Abdi teu ngartos tapi abdi seserian ningali si...
409,joy,Aaaaawww meuni bungah kieu aduh
410,joy,"Aa kasep hatur nuhun, eneng bingah pisan ningg..."


In [103]:
df = pd.concat([train_df, test_df], ignore_index=True)
df

,label,data
0,anger,"sok geura leungit atuh sia teh corona, matak g..."
1,anger,"Nu katoel katuhu nu nyerina kenca, goblog wasitna"
2,joy,Bingah pisan patepang sareng pangerasa. Sing k...
3,fear,asa hariwang kieu.. lalakon hirup teh asa nyor...
4,anger,"Orang mana sih anying, sampis pisan. Bunuh ae ..."
...,...,...
2505,fear,Abi gawe di rs boromeus sararoak kieu:(
2506,joy,Abdi teu ngartos tapi abdi seserian ningali si...
2507,joy,Aaaaawww meuni bungah kieu aduh
2508,joy,"Aa kasep hatur nuhun, eneng bingah pisan ningg..."


In [104]:
df.loc[1]['data']

'Nu katoel katuhu nu nyerina kenca, goblog wasitna'

In [105]:
df

,label,data
0,anger,"sok geura leungit atuh sia teh corona, matak g..."
1,anger,"Nu katoel katuhu nu nyerina kenca, goblog wasitna"
2,joy,Bingah pisan patepang sareng pangerasa. Sing k...
3,fear,asa hariwang kieu.. lalakon hirup teh asa nyor...
4,anger,"Orang mana sih anying, sampis pisan. Bunuh ae ..."
...,...,...
2505,fear,Abi gawe di rs boromeus sararoak kieu:(
2506,joy,Abdi teu ngartos tapi abdi seserian ningali si...
2507,joy,Aaaaawww meuni bungah kieu aduh
2508,joy,"Aa kasep hatur nuhun, eneng bingah pisan ningg..."


In [106]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
df_vectorized = vectorizer.fit_transform(df['data'])
print(vectorizer.get_feature_names_out())

['01' '02' '09' ... 'zuma' 'zumba' 'éta']


In [107]:
df_bow_sklearn = pd.DataFrame(df_vectorized.toarray(),columns=vectorizer.get_feature_names_out())
df_bow_sklearn.head()

,01,02,09,10,100,1285,17,19,1point,20,...,zaman,zamrun,zmn,zohri,zona,zonk,zulham,zuma,zumba,éta
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [108]:
new_df = df.drop(['data'], axis=1)
new_df = pd.concat([new_df, df_bow_sklearn], axis=1)
new_df

,label,01,02,09,10,100,1285,17,19,1point,...,zaman,zamrun,zmn,zohri,zona,zonk,zulham,zuma,zumba,éta
0,anger,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,anger,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,joy,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,fear,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,anger,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2505,fear,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2506,joy,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2507,joy,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2508,joy,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [109]:
label = new_df['label']
new_df = new_df.drop(['label'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(new_df, label, test_size=0.2, random_state=None)

In [110]:
model = DecisionTreeClassifier(criterion='entropy')
model.fit(X_train, y_train)

DecisionTreeClassifier(criterion='entropy')

In [111]:
result = model.predict(X_test)
acc = metrics.accuracy_score(result, y_test)
show = round(acc * 100, 2)
"Accuracy: {}%".format(show)

'Accuracy: 91.43%'